# Re-Analysis of behavioral data and comparison with Kolossa model

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jan 15 20:32:41 2014

@author: moritz
"""

#get the data as np_array of the form chan*time*epochs
from __future__ import division
#re-test for gammas/pnulls
%matplotlib inline
from helper_functions import *

#import re
import os
import pystan
import pandas as pd
import numpy as np
#this is the model
from helper_functions import *
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn.preprocessing import StandardScaler

hierarchic_model = """
data {
int<lower=0> npb;
int<lower=0> cases;
int<lower=0> N[npb,cases];
int<lower=0> y[npb,cases];
real X[cases];
}
parameters {
real<lower=0> mu_gam;
real<lower=0> tau_gam;
real mu_pz;
real<lower=0> tau_pz;
real<lower=0> gam[npb];
real pzero[npb];
}
model {

mu_gam ~ normal(1,1) T[0,];
mu_pz ~ normal(0,10);
gam ~ normal(mu_gam,tau_gam);
pzero ~ normal(mu_pz,tau_pz);


for (i in 1:cases)
{
for (j in 1:npb)
y[j,i] ~ binomial_logit(N[j,i],X[i]*gam[j] + (1-gam[j])*pzero[j]);
}
}
"""

hierarchic_model_kolossa = """
data {
int<lower=0> npb;
int<lower=0> cases;
int<lower=0> N[npb,cases];
int<lower=0> y[npb,cases];
real X[cases];
}
parameters {
real<lower=0> mu_gam;
real<lower=0> tau_gam;
real<lower=0> gam[npb];
}
model {

mu_gam ~ normal(1,1) T[0,];
gam ~ normal(mu_gam,tau_gam);


for (i in 1:cases)
{
for (j in 1:npb)
y[j,i] ~ binomial_logit(N[j,i],X[i]*gam[j]);
}
}
"""

hierarchic_model_kolossa2 = """
data {
int<lower=0> npb;
int<lower=0> cases;
int<lower=0> N[npb,cases];
int<lower=0> y[npb,cases];
real X[cases,4];
}
parameters {
real<lower=0> mu_gam;
real<lower=0> tau_gam;
real<lower=0> gam[npb];
}
model {

mu_gam ~ normal(1,1) T[0,];
gam ~ normal(mu_gam,tau_gam);


for (i in 1:cases)
{
for (j in 1:npb)
y[j,i] ~ bernoulli(inv_logit(gam[j] * x[n,4]+x[n,3]*gam[j]^2+x[n,2]*gam[j]^3+x[n,1]*gam[j]^4));
}
}
"""



k_model = """
data {
int<lower=0> N;
real x[N,4];
int<lower=0,upper=1> y[N];
}
parameters {
real<lower=0> gam;
}
model {

gam ~ normal(1,1) T[0,];

for (n in 1:N)
y[n] ~ bernoulli(inv_logit(gam * x[n,4]+x[n,3]*gam^2+x[n,2]*gam^3+x[n,1]*gam^4));
}
"""

k_model_hierarchic = """
data {
int<lower=0> npb;
int<lower=0> N;
real x[N,4];
int<lower=0,upper=1> y[N];
int pbidx[N];
}
parameters {
real<lower=0> gam[npb];
}
model {
for (n in 1:npb)
    gam[n] ~ normal(1,1) T[0,];

for (i in 1:N)
{
y[i] ~ bernoulli(inv_logit(gam[pbidx[i]] * x[i,4]+x[i,3]*gam[pbidx[i]]^2+x[i,2]*gam[pbidx[i]]^3+x[i,1]*gam[pbidx[i]]^4));
}
}
"""


k_model2 = """
data {
int<lower=0> N;
real x[N,4];
int<lower=0,upper=1> y[N];
}
parameters {
real<lower=0> gam;
real pz;
}
model {

gam ~ normal(1,1) T[0,];
pz ~ normal(0,1.45);
for (n in 1:N)
y[n] ~ bernoulli(inv_logit(gam * x[n,4]+x[n,3]*gam^2+x[n,2]*gam^3+x[n,1]*gam^4+((1-gam)*pz)+((1-gam)*pz)^2+((1-gam)*pz)^3+((1-gam)*pz)^4));
}
"""

In [2]:
#list the files, assuming a folder structure of cwd/Data/...
files = os.listdir("Data/newdata/")

#now get all the data for exp80
pattern = "80exp"

exp80_y = pd.DataFrame({0:14*[0]})
exp80_n = pd.DataFrame({0:14*[0]})

for fn in files:
    if pattern in fn:
        tmp = pd.read_table("Data/newdata/"+fn,sep=" ",header=None)
        exp80_y[int(fn[5:7])] = tmp[0]
        exp80_n[int(fn[5:7])] = tmp[1]
       
#now sort them
exp80_n.sort_index(axis=1,inplace=True)
exp80_y.sort_index(axis=1,inplace=True)

#for easier use convert to array/CHANGE THIS LATER

exp80_n = exp80_n.as_matrix() 
exp80_y = exp80_y.as_matrix()

#and also drop the first column

exp80_n = exp80_n[:,1:]
exp80_y = exp80_y[:,1:]

#pop first 2 cases, so no first ball
exp80_n = exp80_n[2:,:]
exp80_y = exp80_y[2:,:]
#03a or 03?
#now you can start re-analysing the data
#there are 16 vps
npb = 16
#and 14 (12 without 1st ball) different cases for the binomial distribution
cases = 12

#also, we need the actual posterior probabilities as logits


prior = 0.2
likelihood = 0.7

x = np.array([(prior*(likelihood**j)*((1-likelihood)**(i-j)))/(((1-prior)*((likelihood**(i-j))*((1-likelihood)**j)))+(prior*(likelihood**j)*((1-likelihood)**(i-j)))) for i in xrange(1,5) for j in xrange(i+1)])
lo_x = np.log(x/(1-x))
#and again drop first ball/2 cases
lo_x = lo_x[2:]

#transpose so rows are vps and columns are cases
exp80_n = exp80_n.T
exp80_y = exp80_y.T

## Model with 3 balls, non-unique posteriors

In [ ]:
#%%
#ONLY FOR 3 BALLS AND MODEL WITH NON-UNIQUE POSTERIORS
#create the data-structure
data_exp80 = {"npb" : npb,"cases":cases,"N":exp80_n,"y":exp80_y,"X":lo_x}
#fit now
fit = pystan.stan(model_code=hierarchic_model_kolossa, data=data_exp80,iter=10000, chains=10)

## Model with only unique posteriors

In [ ]:
#%%
#MODEL WITH ONLY UNIQUE POSTERIORS 
#try to compress X, so same values are added
#first round

lo_x = around(lo_x,3)

#now add n and y with the same X (with cool comprehension)

exp80_n = array( [ sum(exp80_n[:,lo_x==lx],1) for lx in unique(lo_x)]).T
exp80_y = array( [ sum(exp80_y[:,lo_x==lx],1) for lx in unique(lo_x)]).T

#update cases
cases = exp80_n.shape[1]

data_exp80 = {"npb" : npb,"cases":cases,"N":exp80_n,"y":exp80_y,"X":unique(lo_x)}

In [ ]:
#%%
fit = pystan.stan(model_code=hierarchic_model, data=data_exp80,
                 iter=40000, chains=20)
#%%

## Comparison with Kolossa model

In [3]:
likelihood = { 1 : 0.3, 2: 0.7 }
prior = 0.2

path = "/home/mboos/Work/Bayesian Updating/Data/"
path_mat = "/home/mboos/Work/Bayesian Updating/Data EEG/"

files = os.listdir(path)
mat_files = os.listdir(path_mat)

pattern_TS = "80exp"
pattern = "exp80"

blist_dict = dict()
brar_dict = dict()

#%%
#strip VEOH,HEOG electrodes

#100ms BEFORE stimulus presentation


for fn in files:
    if fn.startswith("TS") and pattern_TS in fn:
        bclass = get_bclass_list(fn,prior,likelihood)
        brar_dict[fn[4:6]] = get_bclass(fn,prior,likelihood)[:,0]
        blist_dict[fn[4:6]] = bclass

 
for pb in sorted(blist_dict.keys()):
    for i,e in enumerate(blist_dict[pb]):
        blist_dict[pb][i][0] = e[0] + e[1]
        blist_dict[pb][i].pop(1)
        blist_dict[pb][i] = [0 for t in xrange(4-len(blist_dict[pb][i]))] + blist_dict[pb][i]

In [4]:
y = np.concatenate([brar_dict[pb]-1 for pb in sorted(brar_dict)])
X = np.vstack([blist_dict[pb] for pb in sorted(brar_dict)])
X = X[y>=0,:]
pbidx = np.concatenate([[int(k)]*brar_dict[k].shape[0] for k in sorted(blist_dict.keys())])
pbidx = pbidx[y>=0]
y = y[y>=0]
y = [int(i) for i in y]

## Load EEG data

In [5]:


def get_coefficients(encoding,X,y,normalize=True,actfilt=True):
    '''Returns coefficients:
    Paramters:
    encoding    -   Encoding (True) or Decoding (False)
    X           -   EEG Timeseries
    y           -   Probability representation
    '''
    if encoding:
        coefs = np.reshape([BayesianRidge(normalize=normalize).fit(y[:,None],xnow).coef_ for xnow in X.T],(5,200))
    else:
        if actfilt:
            yscaler = StandardScaler()
            xscaler = StandardScaler()
            Xcov = np.cov(X,rowvar=0)
            y = yscaler.fit_transform(y)
            X = xscaler.fit_transform(X)
            coefs = BayesianRidge(normalize=False).fit(X,y).coef_
            coefs = np.reshape(Xcov.dot(coefs),(5,200))
        else:
            coefs = np.reshape(BayesianRidge(normalize=normalize).fit(X,y).coef_,(5,200))
    return coefs



chan_list = np.array(["Fp1","Fp2","F7","F3","Fz","F4","F8","FC5","FC1","FC2",\
        "FC6","T7","C3","Cz","C4","T8","CP5","CP1","CP2","CP6","P7","P3","Pz",\
        "P4","P8","TP9","TP10","Oz","O2","O1"])
interesting_ones = ['Fz','FC2','Cz','Pz','Oz']

likelihood = { 1 : 0.3, 2: 0.7 }
prior = 0.2

path = "/home/mboos/Work/Bayesian Updating/Data/"
path_mat = "/home/mboos/Work/Bayesian Updating/Data EEG/"

files = os.listdir(path)
mat_files = os.listdir(path_mat)

pattern_TS = "80exp"
pattern = "exp80"

bc_dict = dict()
mat_dict = dict()
brar_dict = dict()

#%%
#strip VEOH,HEOG electrodes

#100ms BEFORE stimulus presentation


for fn in files:
    if fn.startswith("TS") and pattern_TS in fn:
        f_ep = get_failed_epochs(fn)
        bclass = get_bclass(fn,prior,likelihood)
        #re-reference and average over k bins
        #also think about better way to ensure the result is unique!
        curr = rereference(loadmat(path_mat+filter(lambda x : fn[4:6] in x and pattern in x and "epochs" in x,mat_files)[0])["EEGdata"][:30,25:,:])
        if curr.shape[2] != sum(bclass[:,0]>0):
            continue
        mat_dict[fn[4:6]] = curr[:,:,(f_ep[bclass[:,0]>0])==1]
        bc_dict[fn[4:6]] = bclass[f_ep==1,1]
        brar_dict[fn[4:6]] = bclass[f_ep==1,0]

        


# ### For last two balls

# In[2]:
# ## Different probability representations

# In[5]:

#Kullback-Leibler divergence
nbin = 1#doesnt matter, curr isnt used, only to ensure that we have an estimate of the number of epochs 
kld_dict = dict()
f_ep_dict = dict()

for fn in files:
    if fn.startswith("TS") and pattern_TS in fn:
        f_ep_dict[fn[4:6]] = get_failed_epochs(fn)
        f_ep = f_ep_dict[fn[4:6]]
        kld = kld_vec(fn,prior,likelihood)
        #re-reference and average over k bins
        #also think about better way to ensure the result is unique!
        curr = k_bin_average(rereference(loadmat(path_mat+filter(lambda x : fn[4:6] in x and pattern in x and "epochs" in x,mat_files)[0])["EEGdata"][:30,50:,:]),nbin)
        if curr.shape[2] != sum(kld[:,0]>0):
            continue
        if f_ep.size % 4 != 0:
            print "error at " + fn
            break
        kld_dict[fn[4:6]] = kld[f_ep==1,1]


In [6]:
y_st = np.concatenate([bc_dict[key] for key in sorted(bc_dict.keys())])
X_st = np.vstack([ np.reshape(np.swapaxes(np.swapaxes(mat_dict[key][:,25:-50,:],0,2),1,2)[:,[np.where(chan_list==channel)[0][0] for channel in interesting_ones],:],(-1,1000)) for key in sorted(mat_dict.keys()) ])
xscaler = StandardScaler()
X_st = xscaler.fit_transform(X_st)

In [158]:
k_data = {'N' : X.shape[0],'x':X,'y':y}
fit = pystan.stan(model_code=k_model, data=k_data,iter=5000, chains=10)

/home/mboos/anaconda/lib/python2.7/multiprocessing/queues.py:390: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  return send(obj)
/home/mboos/anaconda/lib/python2.7/multiprocessing/queues.py:390: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  return send(obj)
/home/mboos/anaconda/lib/python2.7/multiprocessing/queues.py:390: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  return send(obj)
/home/mboos/anaconda/lib/python2.7/multiprocessing/queues.py:390: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must b

In [4]:

k_model_hierarchic = """
data {
int<lower=0> npb;
int<lower=0> N;
real x[N,4];
int<lower=0,upper=1> y[N];
int pbidx[N];
}
parameters {
real<lower=0> gam[npb];
}
model {
for (n in 1:npb)
    gam[n] ~ normal(1,1) T[0,];

for (i in 1:N)
{
y[i] ~ bernoulli(inv_logit(gam[pbidx[i]] * x[i,4]+x[i,3]*gam[pbidx[i]]^2+x[i,2]*gam[pbidx[i]]^3+x[i,1]*gam[pbidx[i]]^4));
}
}
"""

k_model_hierarchic_pz = """
data {
int<lower=0> npb;
int<lower=0> N;
real x[N,4];
int<lower=0,upper=1> y[N];
int pbidx[N];
}
parameters {
real<lower=0> gam[npb];
real pz[npb];

}
model {
for (n in 1:npb)
{
    gam[n] ~ normal(1,1) T[0,];
    pz[n] ~ normal(0,1.45);
}

for (n in 1:N)
{
y[n] ~ bernoulli(inv_logit(gam[pbidx[n]] * x[n,4]+x[n,3]*gam[pbidx[n]]^2+x[n,2]*gam[pbidx[n]]^3+x[n,1]*gam[pbidx[n]]^4+((1-gam[pbidx[n]])*pz[pbidx[n]])+((1-gam[pbidx[n]])*pz[pbidx[n]])^2+((1-gam[pbidx[n]])*pz[pbidx[n]])^3+((1-gam[pbidx[n]])*pz[pbidx[n]])^4));
}
}
"""



In [56]:
hierarchic_joint_model = """
data {
int<lower=0> M;
int<lower=0> K;
int<lower=0> npb;
int<lower=0> cases;
int<lower=0> N[npb,cases];
int<lower=0> y[npb,cases];
matrix[M,K] EEG;
vector[M] prob;
matrix[M,npb] idvar;
real X[cases];
}
parameters {
real<lower=0> mu_gam;
real<lower=0> tau_gam;
real mu_pz;
real<lower=0> tau_pz;
vector[npb] gam;
vector[npb] pzero;
vector[K] betas;
real<lower=0> sigma;
}
transformed parameters {
vector[M] unroll_gam;
vector[M] unroll_pz;
vector[M] new_probs;
unroll_gam <- idvar*gam;
unroll_pz <- idvar*pzero;
new_probs <- prob .* unroll_gam + (1-unroll_gam).*unroll_pz;
}

model {

mu_gam ~ normal(1,1) T[0,];
mu_pz ~ normal(0,10);
gam ~ normal(mu_gam,tau_gam);
pzero ~ normal(mu_pz,tau_pz);


for (i in 1:cases)
{
for (j in 1:npb)
y[j,i] ~ binomial_logit(N[j,i],X[i]*gam[j] + (1-gam[j])*pzero[j]);
}

new_probs ~ normal(EEG*betas,sigma);

}
"""

In [83]:
nrs = np.array([mat_dict[k].shape[-1] for k in sorted(mat_dict.keys())])
exp80_n2 = np.delete(exp80_n,13,axis=0)
exp80_y2 = np.delete(exp80_y,13,axis=0)

idvar =  np.vstack([ np.reshape(np.tile(row,nrs[i]),(nrs[i],15)) for i,row in enumerate(np.identity(15))])
data = {'EEG':X_st,'prob':y_st,'K' : X_st.shape[1],'M':X_st.shape[0],'idvar':idvar,"npb" : npb,"cases":cases,"N":exp80_n2,"y":exp80_y2,"X":lo_x}


In [ ]:
fit = pystan.stan(model_code=hierarchic_joint_model, data=data,iter=1000, chains=1)

In [5]:
k_data = {'N' : X.shape[0],'x':X,'y':y,'pbidx':pbidx,'npb':16}
fit = pystan.stan(model_code=k_model_hierarchic_pz, data=k_data,iter=10000, chains=10)

/home/mboos/anaconda/lib/python2.7/multiprocessing/queues.py:392: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  return send(obj)
/home/mboos/anaconda/lib/python2.7/multiprocessing/queues.py:392: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  return send(obj)


RuntimeError: Rejecting initial value:
  Log probability evaluates to log(0), i.e. negative infinity.
  Stan can't start sampling from this initial value.
Rejecting initial value:
  Log probability evaluates to log(0), i.e. negative infinity.
  Stan can't start sampling from this initial value.
Rejecting initial value:
  Log probability evaluates to log(0), i.e. negative infinity.
  Stan can't start sampling from this initial value.
Rejecting initial value:
  Log probability evaluates to log(0), i.e. negative infinity.
  Stan can't start sampling from this initial value.
Rejecting initial value:
  Log probability evaluates to log(0), i.e. negative infinity.
  Stan can't start sampling from this initial value.
Rejecting initial value:
  Log probability evaluates to log(0), i.e. negative infinity.
  Stan can't start sampling from this initial value.
Rejecting initial value:
  Log probability evaluates to log(0), i.e. negative infinity.
  Stan can't start sampling from this initial value.

In [49]:
fit

Inference for Stan model: anon_model_24232061b3deebc8cb4e754770863a5b.
10 chains, each with iter=10000; warmup=5000; thin=1; 
post-warmup draws per chain=5000, total post-warmup draws=50000.

          mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
gam[0]    0.75  2.2e-4   0.05   0.65   0.72   0.75   0.78   0.84  50000    1.0
gam[1]    0.74  2.3e-4   0.05   0.64   0.71   0.74   0.78   0.84  50000    1.0
gam[2]    0.72  2.3e-4   0.05   0.62   0.69   0.73   0.76   0.82  50000    1.0
gam[3]    0.76  2.4e-4   0.05   0.66   0.73   0.77    0.8   0.86  50000    1.0
gam[4]    0.74  2.4e-4   0.05   0.63    0.7   0.74   0.77   0.83  50000    1.0
gam[5]    0.74  2.2e-4   0.05   0.64   0.71   0.74   0.77   0.83  50000    1.0
gam[6]    0.74  2.2e-4   0.05   0.65   0.71   0.75   0.78   0.84  50000    1.0
gam[7]    0.81  2.2e-4   0.05   0.71   0.78   0.81   0.85   0.91  50000    1.0
gam[8]    0.77  2.2e-4   0.05   0.67   0.74   0.77   0.81   0.87  50000    1.0
gam[9]    0.75  2.